In [60]:
from os.path import split

import pandas as pd
import numpy as np
from stravalib import Client
import webbrowser
from Pace import Pace
from runThis import exclude_outliers
from units import *
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import webbrowser
from functions import *
from tqdm import tqdm
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
import ast


# Machine Learning

Objective: analyze pace time and predict predicted pace zones

Predict what types of runs i should do?

In [27]:
client_id = "REDACTED"
client_secret = "REDACTED"

client = setUpClient(client_id, client_secret)

Client works


### Get & Preprocess Data

In [35]:
# USE IF OFFLINE

read = True
myActivities = pd.read_csv('myActivities.csv')
myActivities

,id,run_type,name,start_date,distance,moving_time,elapsed_time,total_elevation_gain,type,average_speed,max_speed,average_heartrate,max_heartrate,granular data
0,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',2.675,7.220,168.4,179.0,"([156, 156, 156, 156, 156, 156, 156, 156, 156,..."
1,13555416250,Tempo,Gotta go fast,2025-02-06 12:40:04+00:00,6050.7,2037,2037,0.0,root='Run',2.970,3.900,171.7,188.0,"([94, 94, 95, 98, 101, 107, 114, 120, 125, 129..."
2,13537636633,Intervals,800m Repeats with Runna ✅,2025-02-04 13:57:26+00:00,9075.6,3472,3506,6.0,root='Run',2.614,5.720,170.1,185.0,"([112, 112, 113, 115, 117, 119, 121, 123, 125,..."
3,13532893683,Free Run,3 mile yap sesh with the club,2025-02-03 23:35:18+00:00,4853.7,2164,2808,7.6,root='Run',2.243,4.280,150.4,171.0,"([171, 171, 171, 171, 171, 171, 171, 171, 171,..."
4,13519770412,Long Run,5.5mi Long Run with Runna ✅,2025-02-02 13:03:40+00:00,8896.1,3416,3928,3.9,root='Run',2.604,4.780,166.8,182.0,"([110, 110, 110, 110, 110, 111, 112, 113, 114,..."
5,13511762315,Intervals,Mile Repeats with Runna ✅,2025-02-01 15:44:36+00:00,9287.5,3294,3389,3.8,root='Run',2.820,5.420,169.0,184.0,"([97, 97, 100, 103, 105, 107, 110, 112, 114, 1..."
6,13502034532,Easy Run,4.5mi Easy Run with Runna ✅,2025-01-31 15:14:33+00:00,5558.5,2089,2089,0.0,root='Run',2.661,3.200,166.9,175.0,"([108, 109, 116, 122, 125, 128, 129, 130, 131,..."
7,13479934531,Time Trial,Red-eye pre game,2025-01-29 00:56:25+00:00,6588.9,2297,2307,31.1,root='Run',2.868,5.620,170.2,185.0,"([143, 143, 143, 143, 143, 143, 143, 143, 143,..."
8,13470342495,Long Run,Lil nostalgic run at home,2025-01-27 22:36:32+00:00,16138.0,6230,6266,112.8,root='Run',2.590,4.320,173.1,183.0,"([121, 121, 122, 124, 126, 127, 129, 128, 126,..."
9,13460371508,Easy Run,Best way to cure the hangover,2025-01-26 18:18:06+00:00,3841.8,1363,1415,28.6,root='Run',2.819,5.180,176.6,191.0,"([159, 163, 164, 160, 158, 155, 153, 150, 148,..."


**Use if you have internet connection**

In [19]:
myActivities = getActivities(client)
read = False
myActivities['granular data'] = myActivities['id'].apply(lambda id: get_activity_streams(client, id, resolution='high'))
old = pd.read_csv('myActivities.csv')
difference = diff(myActivities['name'], old['name'])
distinctTypes = myActivities['run_type'].unique()

# ask for the run_type of run for the new activities and append it to the old dataframe
for nm in difference:
    run_type = None
    activity = myActivities[myActivities['name'] == nm]
    while run_type not in distinctTypes:
        run_type = input(f"Run run_type for {nm}: ")
    assert run_type in distinctTypes, f'Invalid type. Must be one of {distinctTypes}'
    activity['run_type'] = run_type
    old = pd.concat([old, activity])
    old[old['name'] == nm]['run_type'] = run_type
    
old.to_csv('myActivities.csv', index=False)

TypeError: unhashable type: 'RelaxedActivityType'

In [36]:
# hr, pace, elevation, timeIdx, distanceIdx
names = ['HR', 'Pace', 'Elevation', 'TimeIdx', 'DistanceIdx']
for i in range(len(names)):
    name = names[i]
    if read:
        myActivities[name] = myActivities['granular data'].apply(ast.literal_eval).apply(lambda y: y[i])
    else:
        myActivities[name] = myActivities['granular data'].apply(lambda y: y[i])
myActivities.drop('granular data', axis=1, inplace=True)

In [37]:
myActivities['average_speed'] = myActivities['average_speed'].apply(Pace.from_mps)
myActivities['max_speed'] = myActivities['max_speed'].apply(Pace.from_mps)

myActivities = myActivities.explode(names, ignore_index=True)

In [38]:
myActivities['Pace'] = myActivities['Pace'].apply(Pace.from_mps)
myActivities['DistanceIdx'] = myActivities['DistanceIdx'].apply(metersToMiles)

In [39]:
myActivities

,id,run_type,name,start_date,distance,moving_time,elapsed_time,total_elevation_gain,type,average_speed,max_speed,average_heartrate,max_heartrate,HR,Pace,Elevation,TimeIdx,DistanceIdx
0,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,0:00/mi,3.1,0,0.000000
1,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,22:59/mi,3.1,3,0.002175
2,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,20:57/mi,3.1,5,0.003977
3,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,18:24/mi,3.0,7,0.006338
4,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,14:14/mi,2.9,9,0.009196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82591,13131870030,Free Run,Evening Run,2024-11-18 23:36:50+00:00,5201.4,2130,2387,19.0,root='Run',10:59/mi,3:49/mi,165.8,189.0,185,8:49/mi,3.2,2384,3.307940
82592,13131870030,Free Run,Evening Run,2024-11-18 23:36:50+00:00,5201.4,2130,2387,19.0,root='Run',10:59/mi,3:49/mi,165.8,189.0,185,10:43/mi,3.2,2385,3.307940
82593,13131870030,Free Run,Evening Run,2024-11-18 23:36:50+00:00,5201.4,2130,2387,19.0,root='Run',10:59/mi,3:49/mi,165.8,189.0,185,13:41/mi,3.2,2386,3.307940
82594,13131870030,Free Run,Evening Run,2024-11-18 23:36:50+00:00,5201.4,2130,2387,19.0,root='Run',10:59/mi,3:49/mi,165.8,189.0,185,18:53/mi,3.2,2387,3.307940


Make some plots

In [42]:
myActivities['PaceTime'] = myActivities['Pace'].apply(lambda p: p.time)

def splitData(data, column):
    uniqueValues = data[column].unique()
    splitData = []
    for value in uniqueValues:
        splitData.append(data[data[column] == value])
    return splitData

splits = splitData(myActivities, 'run_type')
splits[0]

,id,run_type,name,start_date,distance,moving_time,elapsed_time,total_elevation_gain,type,average_speed,max_speed,average_heartrate,max_heartrate,HR,Pace,Elevation,TimeIdx,DistanceIdx,PaceTime
0,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,0:00/mi,3.1,0,0.000000,0
1,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,22:59/mi,3.1,3,0.002175,1379
2,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,20:57/mi,3.1,5,0.003977,1257
3,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,18:24/mi,3.0,7,0.006338,1104
4,13563730761,Easy Run,Apparently Strava thinks you can run on water,2025-02-07 12:08:38+00:00,5686.2,2126,2264,65.6,root='Run',10:01/mi,3:42/mi,168.4,179.0,156,14:14/mi,2.9,9,0.009196,854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59381,13227317024,Easy Run,2.75mi Easy Run with Runna ✅,2024-12-14 16:43:46+00:00,5659.2,2286,2286,0.0,root='Run',10:49/mi,6:30/mi,170.9,185.0,177,9:59/mi,2.0,2278,3.506282,599
59382,13227317024,Easy Run,2.75mi Easy Run with Runna ✅,2024-12-14 16:43:46+00:00,5659.2,2286,2286,0.0,root='Run',10:49/mi,6:30/mi,170.9,185.0,178,9:30/mi,2.0,2280,3.509762,570
59383,13227317024,Easy Run,2.75mi Easy Run with Runna ✅,2024-12-14 16:43:46+00:00,5659.2,2286,2286,0.0,root='Run',10:49/mi,6:30/mi,170.9,185.0,178,9:38/mi,2.0,2282,3.513304,578
59384,13227317024,Easy Run,2.75mi Easy Run with Runna ✅,2024-12-14 16:43:46+00:00,5659.2,2286,2286,0.0,root='Run',10:49/mi,6:30/mi,170.9,185.0,178,9:48/mi,2.0,2284,3.516473,588


In [57]:
import plotly.express as px
import plotly.graph_objects as go


for split in splits:
    fig = go.Figure()
    run_type = split['run_type'].iloc[0]
    fig.add_trace(go.Scatter(x=split['TimeIdx'], y=split['PaceTime']/60, mode='lines', name=run_type))
    fig.update_layout(title=run_type)
    fig.show()

In [56]:
for split in splits:
    variance = Pace.fromSeconds(split['PaceTime'].var())
    std_dev = Pace.fromSeconds(split['PaceTime'].std())
    mean = Pace.mean(split['PaceTime'].apply(lambda x: Pace.fromSeconds(x)).to_list())
    print(f'\n\nRun Type: {split["run_type"].iloc[0]}')
    print(f'Variance: {variance}')
    print(f'Standard Deviation: {std_dev}')
    print(f'Mean: {mean}')



Run Type: Easy Run
Variance: 16403:57/mi
Standard Deviation: 16:32/mi
Mean: 10:58/mi


Run Type: Tempo
Variance: 11537:01/mi
Standard Deviation: 13:51/mi
Mean: 11:00/mi


Run Type: Intervals
Variance: 51575:46/mi
Standard Deviation: 29:19/mi
Mean: 13:07/mi


Run Type: Free Run
Variance: 252393:21/mi
Standard Deviation: 64:51/mi
Mean: 12:28/mi


Run Type: Long Run
Variance: 16741:56/mi
Standard Deviation: 16:42/mi
Mean: 10:57/mi


Run Type: Time Trial
Variance: 693:34/mi
Standard Deviation: 3:23/mi
Mean: 9:40/mi


In [86]:
def exclude_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3-q1
    lower_fence = q1 - 1.5 * iqr
    upper_fence = q3 + 1.5 * iqr
    return df[(df[column] >= lower_fence) & (df[column] <= upper_fence)]

fig = go.Figure()
for split in splits:
    split = exclude_outliers(split, 'PaceTime')
    
    run_type = split['run_type'].iloc[0]
    
    # make a boxplot of the paces
    fig.add_trace(go.Box(x=split['PaceTime']/60, name=run_type))
fig.update_layout(title='Pace Boxplot', xaxis_title='Pace (min/mi)', yaxis_title='Run Type', showlegend=False, annotations=[
            dict(
                text="Faster <---> Slower",
                x=0.00000001,
                y=1.15,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=12)
            )
        ])
fig.show()

## Predict Pace from Heart Rate and Time Into Run

In [87]:
X = myActivities[['HR', 'TimeIdx']]
y = myActivities['PaceTime']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20)

In [27]:
from sklearn.linear_model import LinearRegression

# Predict Pace from HR and Time Into Run

models = {}
listOfModels = 
for split in splits:
    X = split[['HR', 'TimeIdx']]
    y = split['PaceTime']
    
    
    """X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20)
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    models[split['run_type'].iloc[0]] = model
    print('\n\nrun_type: ', split['run_type'].iloc[0])
    print('Mean Absolute Error:', mean_absolute_error(y_test, predictions))
    print('Mean Squared Error:', mean_squared_error(y_test, predictions))
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, predictions)))"""



Type:  Easy Run
Mean Absolute Error: 159.65041260480456
Mean Squared Error: 664938.7179319462
Root Mean Squared Error: 815.4377462025818


Type:  Tempo
Mean Absolute Error: 230.85699527574207
Mean Squared Error: 134688.8067876009
Root Mean Squared Error: 366.9997367677542


Type:  Intervals
Mean Absolute Error: 342.63075736080515
Mean Squared Error: 3216719.3041319326
Root Mean Squared Error: 1793.5214813689665


Type:  Free Run
Mean Absolute Error: 372.43071372232635
Mean Squared Error: 4398489.640841763
Root Mean Squared Error: 2097.2576477013413


Type:  Long Run
Mean Absolute Error: 150.34803797787524
Mean Squared Error: 3478486.993538228
Root Mean Squared Error: 1865.0702382318548


Type:  Time Trial
Mean Absolute Error: 75.70314381720482
Mean Squared Error: 17356.69868590429
Root Mean Squared Error: 131.7448241332626


In [28]:
# make a plot of the predictions vs the actual values
item = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
item

,Actual,Predicted
10584,643,598.429000
10701,525,644.084221
12002,462,536.697064
11459,481,526.221737
10955,530,502.604606
...,...,...
10644,562,584.544149
11862,574,594.401453
11326,510,485.173413
11531,506,509.959132


In [48]:
random = pd.DataFrame({
    'HR': [167],
    'TimeIdx': [90]
})
predictions = models['Easy Run'].predict(random)
Pace.fromSeconds(predictions[0])

8:53/mi

In [41]:
models['Easy Run'].intercept_, models['Easy Run'].coef_

(np.float64(3207.2156453734397), array([-16.07634569,   0.12754855]))

In [20]:
myActivities['run_type']

0     root='Run'
1     root='Run'
3     root='Run'
4     root='Run'
6     root='Run'
7     root='Run'
8     root='Run'
10    root='Run'
11    root='Run'
12    root='Run'
13    root='Run'
15    root='Run'
16    root='Run'
17    root='Run'
18    root='Run'
19    root='Run'
20    root='Run'
22    root='Run'
24    root='Run'
25    root='Run'
26    root='Run'
27    root='Run'
28    root='Run'
30    root='Run'
32    root='Run'
33    root='Run'
34    root='Run'
37    root='Run'
38    root='Run'
39    root='Run'
40    root='Run'
41    root='Run'
42    root='Run'
43    root='Run'
44    root='Run'
45    root='Run'
46    root='Run'
47    root='Run'
48    root='Run'
49    root='Run'
50    root='Run'
51    root='Run'
52    root='Run'
53    root='Run'
54    root='Run'
55    root='Run'
56    root='Run'
57    root='Run'
58    root='Run'
59    root='Run'
60    root='Run'
Name: type, dtype: object